In [1]:
import torch.nn
import torch.utils.data as data
import os
from PIL import Image
import numpy as np
from torchvision import transforms

In [7]:
# for CRNN
class Dataset_CRNN(data.Dataset):
    "Characterizes a dataset for PyTorch"
    def __init__(self, data_path, frame_length=10, transform=None):
        "Initialization"
        self.transform = transform
        #self.frames = frames
        self.folders = data_path
        self.frames = frame_length #For our case since we are computing 10 frames always

    def __len__(self):
        "Denotes the total number of samples"
        return len(os.listdir(self.folders))

    def read_images(self, data_path, use_transform):
        X = []
        file_name = ""
        for i in os.listdir(data_path):
            file_name = i
            image = Image.open(os.path.join(data_path,i))
            
            #print(image.shape)
            if use_transform is not None:
                image = use_transform(image)
                #print(image.size)
            image = torch.from_numpy(np.asarray(image))
            X.append(image)
        X = torch.stack(X, dim=0)

        return X, file_name

    def __getitem__(self, index):
        data_path = os.path.join(self.folders,os.listdir(self.folders)[index])
              
        # Load data
        X, file_name = self.read_images(data_path, self.transform)                     # (input) spatial images
        
        y = np.ones(self.frames)
        if 'real' in data_path:
            y = np.zeros(self.frames)
        #print("Folder is {}".format(data_path))
        #print(X.shape)
        return X, torch.from_numpy(y).type(torch.LongTensor)

In [9]:
TRANSFORM_IMG = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(256),
    transforms.ToTensor(),
    #transforms.ToTensor()
    #transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         #std=[0.229, 0.224, 0.225] )
    ])

In [10]:
train_path = '/home/chinmay/datatset/train/'
train_data = Dataset_CRNN(train_path, transform=TRANSFORM_IMG)
# for step, (x, y) in enumerate(data):
#     print(x.shape)
#val_path = "/home/chinmay/datatset/deepfake_split/val"
#val_data = Dataset_CRNN(val_path, transform=TRANSFORM_IMG)

In [11]:
loader = data.DataLoader(
    train_data,
    batch_size=8,
    num_workers=1,
    shuffle=False
)

In [12]:
for batch_idx, (X, y) in enumerate(loader):
    print(X.shape)

torch.Size([8, 10, 3, 256, 256])
torch.Size([8, 10, 3, 256, 256])
torch.Size([8, 10, 3, 256, 256])
torch.Size([8, 10, 3, 256, 256])
torch.Size([8, 10, 3, 256, 256])
torch.Size([8, 10, 3, 256, 256])
torch.Size([8, 10, 3, 256, 256])
torch.Size([8, 10, 3, 256, 256])
torch.Size([8, 10, 3, 256, 256])
torch.Size([8, 10, 3, 256, 256])
torch.Size([8, 10, 3, 256, 256])
torch.Size([8, 10, 3, 256, 256])
torch.Size([8, 10, 3, 256, 256])
torch.Size([8, 10, 3, 256, 256])
torch.Size([8, 10, 3, 256, 256])
torch.Size([8, 10, 3, 256, 256])
torch.Size([8, 10, 3, 256, 256])
torch.Size([8, 10, 3, 256, 256])
torch.Size([8, 10, 3, 256, 256])
torch.Size([8, 10, 3, 256, 256])
torch.Size([8, 10, 3, 256, 256])
torch.Size([8, 10, 3, 256, 256])
torch.Size([8, 10, 3, 256, 256])
torch.Size([8, 10, 3, 256, 256])
torch.Size([8, 10, 3, 256, 256])
torch.Size([8, 10, 3, 256, 256])
torch.Size([8, 10, 3, 256, 256])
torch.Size([8, 10, 3, 256, 256])
torch.Size([8, 10, 3, 256, 256])
torch.Size([8, 10, 3, 256, 256])
torch.Size

In [13]:
mean = 0.
std = 0.
nb_samples = 0.
for data_iter in loader:
    data = data_iter[0]
    batch_samples = data.size(0)
    # get the data seize
    # print(data.size())
    data = data.view(batch_samples, data.size(2), -1)
    print(data.size())
    mean += data.mean(2).sum(0)
    std += data.std(2).sum(0)
    nb_samples += batch_samples

torch.Size([8, 3, 655360])
torch.Size([8, 3, 655360])
torch.Size([8, 3, 655360])
torch.Size([8, 3, 655360])
torch.Size([8, 3, 655360])
torch.Size([8, 3, 655360])
torch.Size([8, 3, 655360])
torch.Size([8, 3, 655360])
torch.Size([8, 3, 655360])
torch.Size([8, 3, 655360])
torch.Size([8, 3, 655360])
torch.Size([8, 3, 655360])
torch.Size([8, 3, 655360])
torch.Size([8, 3, 655360])
torch.Size([8, 3, 655360])
torch.Size([8, 3, 655360])
torch.Size([8, 3, 655360])
torch.Size([8, 3, 655360])
torch.Size([8, 3, 655360])
torch.Size([8, 3, 655360])
torch.Size([8, 3, 655360])
torch.Size([8, 3, 655360])
torch.Size([8, 3, 655360])
torch.Size([8, 3, 655360])
torch.Size([8, 3, 655360])
torch.Size([8, 3, 655360])
torch.Size([8, 3, 655360])
torch.Size([8, 3, 655360])
torch.Size([8, 3, 655360])
torch.Size([8, 3, 655360])
torch.Size([8, 3, 655360])
torch.Size([8, 3, 655360])
torch.Size([8, 3, 655360])
torch.Size([8, 3, 655360])
torch.Size([8, 3, 655360])
torch.Size([8, 3, 655360])
torch.Size([8, 3, 655360])
t

In [14]:
mean /= nb_samples
std /= nb_samples

In [15]:
print("Mean of the sample is {}".format(mean))
print("Standard Deviation {}".format(std))

Mean of the sample is tensor([0.4246, 0.4144, 0.4114])
Standard Deviation tensor([0.2265, 0.2208, 0.2215])


#    # Finding the Best Predicted image on validation set

In [88]:
TRANSFORM_IMG = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(256),
    #transforms.ToTensor()
    #transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         #std=[0.229, 0.224, 0.225] )
    ])

In [89]:
val_path = '/home/chinmay/datatset/deepfake_split/val'
val_data = Dataset_CRNN(val_path, transform=TRANSFORM_IMG, frame_length=30)

In [95]:
loss_function = torch.nn.CrossEntropyLoss()

In [96]:
use_cuda = True
params = {'batch_size': 8, 'shuffle': True, 'num_workers': 4, 'pin_memory': True} if use_cuda else {}
valid_loader = data.DataLoader(val_data, **params)

In [113]:
def find_median(numpy_array = []): #This is expected to take an array of array. So,
    #print("Input array is {}".format(numpy_array))
    output = []
    for array in numpy_array:
        counts = np.bincount(array)
        output.append(np.argmax(counts))
    return torch.from_numpy(np.asarray(output)).type(torch.LongTensor)

In [114]:
def validation(model, device, test_loader):
    # set model as testing mode
    model.eval()

    test_loss = 0
    all_y = []
    all_y_pred = []
    max_score = 0
    max_label = ''
    with torch.no_grad():
        for X, y,file_name in test_loader:
            # distribute data to device
            X, y = X.to(device), y.to(device)
            #X = X.permute(0,1,4,2,3)# Required to match shapes
            #print(X.shape)
            output = model(X)
            #print("Output shape is {}".format(output.shape))
            #print("Expected shape is {}".format(y.shape))
            for items in range(output.shape[1]):
                loss = loss_function(output[:,items,:], y[:,items])
                test_loss += loss.item()                 # sum up batch loss
            #Predict value
            
            y_pred = torch.max(output, 2)[1]  # (y_pred != output) get the index of the max log-probability
            # Replace the value with the median value
            possible_scores = torch.max(output, 2)[0]
            #print(possible_scores)
            for score_arr in possible_scores:
                for score in score_arr:
                    if score > max_score:
                        print("score array is {}".format(score_arr))
                        print("And filename is {}".format(file_name))
                        max_score = score     
                        #index = X[score_arr == score] #Getting from X the value we wanted
                        # max_label = index
            y_pred = find_median(y_pred)
            y = find_median(y)
            print("actual value of labels read from disc is {}".format(y))
            print("filename is {} and prediction is {}".format(file_name, y_pred))
            # collect all y and y_pred in all batches
            all_y.extend(y)
            all_y_pred.extend(y_pred)
    print("maximum score is {}".format(max_score))
    test_loss /= len(test_loader.dataset)

    # compute accuracy
#     all_y = torch.stack(all_y, dim=0)
#     all_y_pred = torch.stack(all_y_pred, dim=0)
#     test_score = accuracy_score(all_y.cpu().data.squeeze().numpy(), all_y_pred.cpu().data.squeeze().numpy())

#     # show information
#     print('\nTest set ({:d} samples): Average loss: {:.4f}, Accuracy: {:.2f}%\n'.format(len(all_y), test_loss, 100* test_score))

    # save Pytorch models of best record
    #   torch.save(optimizer.state_dict(), os.path.join(save_model_path, 'optimizer_epoch{}.pth'.format(epoch + 1)))      # save optimizer
#     print("Epoch {} model saved!".format(epoch + 1))

    return 0, 0 #Immaterial values

In [115]:
from Meso4 import Meso4
model = Meso4()

# Load data from the specified location
save_path = "/home/chinmay/visualisation/pytorch-cnn-visualizations/src/"
PATH = save_path + "meso_df_2_epoch28.pth"
 
model.load_state_dict(torch.load(PATH))
model.cuda()
epochs = 1

In [116]:
for epoch in range(epochs):
    # train, test model
    # train_losses, train_scores = train(log_interval, model, device, train_loader, optimizer, epoch)
    epoch_test_loss, epoch_test_score = validation(model, 'cuda', valid_loader)
    # Reduce learning-rate by a factor of 1/10 after every 10 epochs
    # avoid this step as Adam is being used
    #adjust_learning_rate(optimizer=optimizer, learning_rate=learning_rate, epoch=epoch)
    
    torch.cuda.empty_cache()


score array is tensor([2.5441, 2.9198, 1.6562, 2.4362, 2.4476, 2.9700, 2.0143, 2.3608, 1.9688,
        0.9938, 1.8933, 2.3053, 1.6846, 1.1032, 0.1836, 2.2953, 1.2419, 0.5873,
        2.3343, 0.3434, 1.4414, 2.3553, 2.5622, 1.7238, 0.3480, 0.8488, 2.5494,
        2.0040, 1.3833, 0.2105], device='cuda:0')
And filename is ['132_228.jpg', '78_828.jpg', '58_138.jpg', '92_714.jpg', '128_1122.jpg', '68_664.jpg', '99_298.jpg', '175_12.jpg']
score array is tensor([2.5441, 2.9198, 1.6562, 2.4362, 2.4476, 2.9700, 2.0143, 2.3608, 1.9688,
        0.9938, 1.8933, 2.3053, 1.6846, 1.1032, 0.1836, 2.2953, 1.2419, 0.5873,
        2.3343, 0.3434, 1.4414, 2.3553, 2.5622, 1.7238, 0.3480, 0.8488, 2.5494,
        2.0040, 1.3833, 0.2105], device='cuda:0')
And filename is ['132_228.jpg', '78_828.jpg', '58_138.jpg', '92_714.jpg', '128_1122.jpg', '68_664.jpg', '99_298.jpg', '175_12.jpg']
score array is tensor([2.5441, 2.9198, 1.6562, 2.4362, 2.4476, 2.9700, 2.0143, 2.3608, 1.9688,
        0.9938, 1.8933, 2.3053,

actual value of labels read from disc is tensor([1, 0, 0, 0, 0, 0, 0, 0])
filename is ['115_6.jpg', '84_273.jpg', '87_900.jpg', '86_362.jpg', '78_741.jpg', '88_40.jpg', '67_534.jpg', '48_240.jpg'] and prediction is tensor([1, 0, 0, 0, 0, 0, 0, 0])
actual value of labels read from disc is tensor([1, 1, 1, 0, 1, 0, 0, 1])
filename is ['163_188.jpg', '152_316.jpg', '146_252.jpg', '49_444.jpg', '175_68.jpg', '57_66.jpg', '53_360.jpg', '162_128.jpg'] and prediction is tensor([1, 1, 1, 0, 1, 0, 0, 1])
actual value of labels read from disc is tensor([1, 1, 1, 1, 0, 0, 0, 0])
filename is ['125_2.jpg', '164_82.jpg', '148_0.jpg', '168_110.jpg', '88_161.jpg', '82_96.jpg', '76_138.jpg', '96_66.jpg'] and prediction is tensor([1, 1, 1, 1, 1, 0, 0, 0])
actual value of labels read from disc is tensor([0, 0, 0, 0, 0, 0, 0, 0])
filename is ['91_252.jpg', '44_72.jpg', '100_824.jpg', '94_552.jpg', '65_82.jpg', '74_816.jpg', '91_480.jpg', '77_99.jpg'] and prediction is tensor([0, 0, 0, 0, 0, 0, 0, 0])
scor

In [ ]:
# Loader for the faceForensic datatset to see the best image

In [117]:
val_path = '/home/chinmay/datatset/val'
val_data = Dataset_CRNN(val_path, transform=TRANSFORM_IMG, frame_length=10)

In [118]:
from Meso4 import Meso4
model = Meso4()

# Load data from the specified location
save_path = "/home/chinmay/visualisation/pytorch-cnn-visualizations/src/"
PATH = save_path + "meso_f2f_epoch28.pth"
 
model.load_state_dict(torch.load(PATH))
model.cuda()
epochs = 1

In [120]:
use_cuda = True
params = {'batch_size': 8, 'shuffle': True, 'num_workers': 4, 'pin_memory': True} if use_cuda else {}
valid_loader = data.DataLoader(val_data, **params)

In [121]:
for epoch in range(epochs):
    # train, test model
    # train_losses, train_scores = train(log_interval, model, device, train_loader, optimizer, epoch)
    epoch_test_loss, epoch_test_score = validation(model, 'cuda', valid_loader)
    # Reduce learning-rate by a factor of 1/10 after every 10 epochs
    # avoid this step as Adam is being used
    #adjust_learning_rate(optimizer=optimizer, learning_rate=learning_rate, epoch=epoch)
    
    torch.cuda.empty_cache()

score array is tensor([0.9196, 1.0822, 2.3597, 0.8591, 1.3110, 2.2409, 0.7416, 0.1697, 0.5236,
        0.3696], device='cuda:0')
And filename is ['yA9PpKq7EMY_1_y4PqkuhOzNc_1_original_input_5.png', 'qpBoMZ8mAiQ_2_qpe2zydDOBM_0_3.png', 'uuzCB0gt8kM_0_wnx2fsN9WP0_1_6.png', 'unJBUjkCJ-s_0_uuCpdpU7Ny0_0_original_input_8.png', 'rIiQLEa8iZ0_2_r4evQe47qE0_4_original_input_6.png', 'qQUcjcS06IY_1_qs8SVi4pA4Y_1_6.png', 'xl0CyL44YzA_1_yq2rKzQrYVg_2_original_input_7.png', 's8raRFJ7z9w_0_sqfDnCME350_0_9.png']
score array is tensor([0.9196, 1.0822, 2.3597, 0.8591, 1.3110, 2.2409, 0.7416, 0.1697, 0.5236,
        0.3696], device='cuda:0')
And filename is ['yA9PpKq7EMY_1_y4PqkuhOzNc_1_original_input_5.png', 'qpBoMZ8mAiQ_2_qpe2zydDOBM_0_3.png', 'uuzCB0gt8kM_0_wnx2fsN9WP0_1_6.png', 'unJBUjkCJ-s_0_uuCpdpU7Ny0_0_original_input_8.png', 'rIiQLEa8iZ0_2_r4evQe47qE0_4_original_input_6.png', 'qQUcjcS06IY_1_qs8SVi4pA4Y_1_6.png', 'xl0CyL44YzA_1_yq2rKzQrYVg_2_original_input_7.png', 's8raRFJ7z9w_0_sqfDnCME350_0_9.pn

actual value of labels read from disc is tensor([1, 1, 1, 1, 1, 1, 1, 1])
filename is ['szxR2kMIWtI_0_saQyF1FPiqw_0_3.png', 'viq-Tn_w1MM_0_wCIDKwgcpfw_0_original_input_7.png', 'zRUomg6M3j0_0_zPdvu-SUnbg_1_9.png', 'yuyl1f-B6iU_0_y4PqkuhOzNc_1_original_input_7.png', 'r4evQe47qE0_4_rGszRN55VHs_3_original_input_4.png', 'vRCCZzmv5xs_1_ugyYHAxIMrc_0_7.png', 'x4dffEgdvwU_1_yX3GBqrETss_1_3.png', 'zA9-nGZacyg_0_zD_OLZWppaA_1_8.png'] and prediction is tensor([1, 0, 1, 0, 0, 1, 1, 1])
actual value of labels read from disc is tensor([1, 1, 1, 1, 1, 1, 1, 1])
filename is ['vJv2Bemw7MU_4_r0WUkAPxFhw_8_original_input_8.png', 'r0WUkAPxFhw_8_wAu_YWTQ3Po_0_original_input_2.png', 'w9H1lP-eOoA_3_wedLZCKQSrM_2_3.png', 'xffuIgiRUZw_6_ySy2dIJ889c_0_original_input_3.png', 'qZt442xDsAw_0_r4evQe47qE0_3_original_input_0.png', 'szxR2kMIWtI_1_tTzFkD7YJ8g_2_6.png', 'zIowgsPebcE_7_zR3A0fzf1Kg_1_original_input_4.png', 'xG9J__APvnY_8_x99NFdLvAIQ_1_original_input_1.png'] and prediction is tensor([0, 0, 1, 0, 0, 1, 0, 1

score array is tensor([2.7327, 2.7399, 1.7907, 1.0890, 2.9628, 2.9555, 1.3655, 3.6005, 2.0438,
        1.6391], device='cuda:0')
And filename is ['xQHrfDT6XJ0_0_ySy2dIJ889c_0_0.png', 'qZt442xDsAw_0_r4evQe47qE0_3_5.png', 'xl0CyL44YzA_1_yq2rKzQrYVg_2_2.png', 'wedLZCKQSrM_0_w2h4xISxMtc_2_1.png', 'yd17VGQZJs8_0_yj04mRa-AMA_1_9.png', 'tApBREQnlcc_3_t6rrtC12SfM_2_3.png', 'xZjp2wTRB00_3_yueeQKJK-wc_0_0.png', 't2T_pvkLm3s_1_s8CuBT-MDYE_4_original_input_7.png']
actual value of labels read from disc is tensor([1, 1, 1, 1, 1, 1, 1, 1])
filename is ['xQHrfDT6XJ0_0_ySy2dIJ889c_0_0.png', 'qZt442xDsAw_0_r4evQe47qE0_3_5.png', 'xl0CyL44YzA_1_yq2rKzQrYVg_2_2.png', 'wedLZCKQSrM_0_w2h4xISxMtc_2_1.png', 'yd17VGQZJs8_0_yj04mRa-AMA_1_9.png', 'tApBREQnlcc_3_t6rrtC12SfM_2_3.png', 'xZjp2wTRB00_3_yueeQKJK-wc_0_0.png', 't2T_pvkLm3s_1_s8CuBT-MDYE_4_original_input_7.png'] and prediction is tensor([0, 1, 1, 1, 0, 1, 0, 0])
actual value of labels read from disc is tensor([1, 1, 1, 1, 1, 1, 1, 1])
filename is ['wlSJLD